In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Chat 모델 및 프롬프트 설정
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature= 0.1, # 낮을수록 일관된 답변, 높을수록 창의적인 답변
    streaming=True, # 답변이 생성되는 과정을 실시간으로 확인
    callbacks=[StreamingStdOutCallbackHandler()] 
)   

# Few-Shot Learning을 위한 예제 데이터
# AI에게 원하는 답변 형식을 학습시키기 위한 예시들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 각 예제를 어떤 형식으로 보여줄지 정의
example_template = """
    Human: {question}
    AI: {answer}
"""

# 예제 템플릿을 PromptTemplate 객체로 변환
example_prompt = PromptTemplate.from_template(example_template)

# FewShotPromptTemplate: 여러 예제와 함께 프롬프트를 구성
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt, # 예제 형식 템플릿
    examples = examples, # 실제 예제 데이터
    suffix = "Human: What do you know about {country}?", # 예제 뒤에 붙는 실제 질문
    input_variables=["country"] # 실제 질문 템플릿(suffix)에서 사용할 입력 변수 (유효성 검사용)
)

# LangChain Expression Language를 사용할 체인 구성
chain = prompt | chat

# 체인 실행: Korea에 대한 정보 요청
chain.invoke({
    "country": "Korea"
})

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')